# Weatherdata for IPM Plateform

## 1. import weather data modules

In [1]:
from weatherdata.ipm import WeatherDataHub 

## 2. Access to weatherdatasource available on the platefom

In [2]:
ws=WeatherDataHub()

In [3]:
ws.list_resources()

{'Met Norway Locationforecast': '9-day forecasts for the entire planet. 2.5 km resolution in the Nordic-Baltic region, 9km elsewhere',
 'Deutsche Wetterdienst location forecast by IPM Decisions': "27 hour weather forecasts for Germany and surrounding areas. 2.5 km resolution. Pulls data from DWD's Open Data server (https://www.dwd.de/EN/ourservices/opendata/opendata.html). Made available by a custom adapter created by Met Norway and NIBIO.",
 'Deutsche Wetterdienst EU Area location forecast by IPM Decisions': "30 hour weather forecasts for EU. 7 km resolution. Pulls data from DWD's Open Data server (https://www.dwd.de/EN/ourservices/opendata/opendata.html). Made available by a custom adapter created by Met Norway and NIBIO.",
 'MeteoFrance location forecast by IPM Decisions': "40 hour weather forecasts for France and surroundings. 2.5 km resolution. Pulls data from MeteoFrance's Public Data service (https://donneespubliques.meteofrance.fr). Made available by a custom adapter created by

## 3. Get ressource for a specific weatherdataresource 

To connect to the meteo resource, simply enter the name of the meteo resource in the get_ressource function. 

In [4]:
fmi=ws.get_ressource(name='Finnish Meteorological Institute measured data')

Once connected to the resource various functions are available. You can check 
* the available stations for the resource with station_ids function. This function This function returns a dataframe of the stations available for the resource with their names, their identifiers and the coordinates of the station (latitude, longitude)
* the weather parameters accepted by the resource  with parameters function. This function return common and optional weather parameter 
* Get Weatherdata in xarray dataset with attribute or json format (more description of this function below)

In [5]:
fmi.station_ids()

,name,id,latitude,longitude
0,Alajärvi Möksy,101533,24.26,63.09
1,Asikkala Pulkkilanharju,101185,25.52,61.27
2,Enontekiö Kilpisjärvi Saana,102017,20.85,69.04
3,Enontekiö Kilpisjärvi,102016,20.79,69.05
4,Enontekiö Näkkälä,102019,23.58,68.6
...,...,...,...,...
203,Virolahti Koivuniemi,101231,27.67,60.53
204,Virrat Äijänneva,101310,23.54,62.33
205,Ylitornio Meltosjärvi,101908,24.65,66.53
206,Ylivieska airfield,101690,24.72,64.05


In [6]:
fmi.parameters()

{'common': [1002, 3002, 2001, 4003], 'optional': None}

### Get weather data

According to weather resources differents parameters can be used:
* *for historic weatherdata*
    * parameters: list of weather parameters available for the resource selected
    * station_id: list of station id available for the resource selected
    * timeStart: Start date of the request
    * timeEnd: End date of the request
    * timezone: The timezone
    * format: 'ds' to obtain a xarray dataset or 'json' to obtain the json     
    
    
* *for forecasts weather resources*
    * latitude: list of latitude
    * longitude: list of longitude
    * altitude: list of altitude

#### Example for historic weather ressources 
* for one station_id

In [14]:
# for one station_ids
ds=fmi.data(parameters=[1002,3002],
            stationId=[101533],
            timeStart='2020-06-12',
            timeEnd='2020-07-03',
            timeZone='UTC',
            format='ds')
ds

%(levelname)s:%(message)s


<xarray.Dataset>
Dimensions:   (alt: 1, lat: 1, location: 1, lon: 1, time: 505)
Coordinates:
  * time      (time) datetime64[ns] 2020-06-12 ... 2020-07-03
  * location  (location) int32 101533
  * lat       (lat) float64 63.09
  * lon       (lon) float64 24.26
  * alt       (alt) float64 0.0
Data variables:
    1002      (time, location) float64 4.8 4.0 4.2 9.5 ... 14.0 11.7 11.1 11.3
    3002      (time, location) float64 86.0 90.0 91.0 81.0 ... 89.0 96.0 97.0
Attributes:
    weatherRessource:  Finnish Meteorological Institute measured data
    weatherStationId:  [101533]
    longitude:         [24.26084]
    latitude:          [63.08898]
    timeStart:         2020-06-12T00:00:00.000000000
    timeEnd:           2020-07-03T00:00:00.000000000
    parameters:        ['1002', '3002']

In [16]:
ds.to_dataframe()

1002  3002
alt lat      location lon      time                           
0.0 63.08898 101533   24.26084 2020-06-12 00:00:00   4.8  86.0
                               2020-06-12 01:00:00   4.0  90.0
                               2020-06-12 02:00:00   4.2  91.0
                               2020-06-12 03:00:00   9.5  81.0
                               2020-06-12 04:00:00  13.0  71.0
...                                                  ...   ...
                               2020-07-02 20:00:00  13.9  58.0
                               2020-07-02 21:00:00  14.0  69.0
                               2020-07-02 22:00:00  11.7  89.0
                               2020-07-02 23:00:00  11.1  96.0
                               2020-07-03 00:00:00  11.3  97.0

[505 rows x 2 columns]

* for several station_ids

In [17]:
ds=fmi.data(parameters=[1002,3002],
            stationId=[101533,101185],
            timeStart='2020-06-12',
            timeEnd='2020-07-03',
            timeZone='UTC',
            format='ds')
ds

%(levelname)s:%(message)s
%(levelname)s:%(message)s


<xarray.Dataset>
Dimensions:   (alt: 1, lat: 2, location: 2, lon: 2, time: 505)
Coordinates:
  * lat       (lat) float64 61.27 63.09
  * location  (location) int64 101185 101533
  * time      (time) datetime64[ns] 2020-06-12 ... 2020-07-03
  * lon       (lon) float64 24.26 25.52
  * alt       (alt) float64 0.0
Data variables:
    1002      (lon, lat, time, location) float64 nan nan nan nan ... nan nan nan
    3002      (lon, lat, time, location) float64 nan nan nan nan ... nan nan nan
Attributes:
    weatherRessource:  Finnish Meteorological Institute measured data
    weatherStationId:  [101533, 101185]
    longitude:         [24.26084, 25.52021]
    latitude:          [61.26521, 63.08898]
    timeStart:         2020-06-12T00:00:00.000000000
    timeEnd:           2020-07-03T00:00:00.000000000
    parameters:        ['1002', '3002']

In [19]:
ds.to_dataframe().dropna()

1002  3002
alt lat      location lon      time                           
0.0 61.26521 101185   25.52021 2020-06-12 00:00:00  13.4  82.0
                               2020-06-12 01:00:00  14.0  77.0
                               2020-06-12 02:00:00  15.1  70.0
                               2020-06-12 03:00:00  14.8  69.0
                               2020-06-12 04:00:00  15.6  66.0
...                                                  ...   ...
    63.08898 101533   24.26084 2020-07-02 20:00:00  13.9  58.0
                               2020-07-02 21:00:00  14.0  69.0
                               2020-07-02 22:00:00  11.7  89.0
                               2020-07-02 23:00:00  11.1  96.0
                               2020-07-03 00:00:00  11.3  97.0

[1010 rows x 2 columns]

#### For forecast weather resources
* example for one latitude, longitude, altitude

In [20]:
norway=ws.get_ressource(name='Met Norway Locationforecast')

In [21]:
ds=norway.data(latitude=[67.2828], longitude=[14.3711], altitude=[70],format='ds')
ds

<xarray.Dataset>
Dimensions:   (alt: 1, lat: 1, location: 1, lon: 1, time: 223)
Coordinates:
  * time      (time) datetime64[ns] 2021-10-08T12:00:00 ... 2021-10-17T18:00:00
  * location  (location) <U18 '[67.2828, 14.3711]'
  * lat       (lat) float64 67.28
  * lon       (lon) float64 14.37
  * alt       (alt) float64 70.0
Data variables:
    1001      (time, location) float64 8.2 8.6 11.2 11.3 ... 6.633 6.617 6.6
    3001      (time, location) float64 89.1 93.6 100.0 ... 84.07 85.68 87.3
    2001      (time, location) float64 1.2 1.4 0.3 0.2 0.4 ... 0.0 0.0 0.0 0.0
    4002      (time, location) float64 5.8 4.4 9.2 8.5 ... 5.75 6.167 6.583 7.0
Attributes:
    weatherRessource:  Met Norway Locationforecast
    longitude:         [14.3711]
    latitude:          [67.2828]
    timeStart:         2021-10-08T12:00:00.000000000
    timeEnd:           2021-10-17T18:00:00.000000000
    parameters:        ['1001', '3001', '2001', '4002']

In [22]:
ds.to_dataframe()

1001  \
alt  lat     location           lon     time                             
70.0 67.2828 [67.2828, 14.3711] 14.3711 2021-10-08 12:00:00   8.200000   
                                        2021-10-08 13:00:00   8.600000   
                                        2021-10-08 14:00:00  11.200000   
                                        2021-10-08 15:00:00  11.300000   
                                        2021-10-08 16:00:00  11.300000   
...                                                                ...   
                                        2021-10-17 14:00:00   6.666667   
                                        2021-10-17 15:00:00   6.650000   
                                        2021-10-17 16:00:00   6.633333   
                                        2021-10-17 17:00:00   6.616667   
                                        2021-10-17 18:00:00   6.600000   

                                                                   3001  2001  \
alt  lat     location           lon     time                                    
70.0 67.2828 [67.2828, 14.3711] 14.3711 2021-10-08 12:00:00   89.100000   1.2   
                                        2021-10-08 13:00:00   93.600000   1.4   
                                        2021-10-08 14:00:00  100.000000   0.3   
                                        2021-10-08 15:00:00  100.000000   0.2   
                                        2021-10-08 16:00:00   99.900000   0.4   
...                                                                 ...   ...   
                                        2021-10-17 14:00:00   80.833333   0.0   
                                        2021-10-17 15:00:00   82.450000   0.0   
                                        2021-10-17 16:00:00   84.066667   0.0   
                                        2021-10-17 17:00:00   85.683333   0.0   
                                        2021-10-17 18:00:00   87.300000   0.0   

                                                                 4002  
alt  lat     location           lon     time                           
70.0 67.2828 [67.2828, 14.3711] 14.3711 2021-10-08 12:00:00  5.800000  
                                        2021-10-08 13:00:00  4.400000  
                                        2021-10-08 14:00:00  9.200000  
                                        2021-10-08 15:00:00  8.500000  
                                        2021-10-08 16:00:00  9.200000  
...                                                               ...  
                                        2021-10-17 14:00:00  5.333333  
                                        2021-10-17 15:00:00  5.750000  
                                        2021-10-17 16:00:00  6.166667  
                                        2021-10-17 17:00:00  6.583333  
                                        2021-10-17 18:00:00  7.000000  

[223 rows x 4 columns]

In [ ]:
ds=norway.data(latitude=[67.2828,61.27], longitude=[14.3711,25.52], altitude=[70, 0],format='ds')
ds

**From ds you can see data as dataframe with function to_dataframe and exclude na value with dropna**
for more information on the http://xarray.pydata.org/en/stable/index.html

In [ ]:
ds.to_dataframe().dropna()

# Metpy 

In [25]:
import metpy.calc as mpcalc
from metpy.units import units
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import xarray as xr

In [26]:
ds=fmi.data(parameters=[1002,3002],
            stationId=[101533],
            timeStart='2020-06-12',
            timeEnd='2020-07-03',
            timeZone='UTC',
            format='ds')
ds

%(levelname)s:%(message)s


<xarray.Dataset>
Dimensions:   (alt: 1, lat: 1, location: 1, lon: 1, time: 505)
Coordinates:
  * time      (time) datetime64[ns] 2020-06-12 ... 2020-07-03
  * location  (location) int32 101533
  * lat       (lat) float64 63.09
  * lon       (lon) float64 24.26
  * alt       (alt) float64 0.0
Data variables:
    1002      (time, location) float64 4.8 4.0 4.2 9.5 ... 14.0 11.7 11.1 11.3
    3002      (time, location) float64 86.0 90.0 91.0 81.0 ... 89.0 96.0 97.0
Attributes:
    weatherRessource:  Finnish Meteorological Institute measured data
    weatherStationId:  [101533]
    longitude:         [24.26084]
    latitude:          [63.08898]
    timeStart:         2020-06-12T00:00:00.000000000
    timeEnd:           2020-07-03T00:00:00.000000000
    parameters:        ['1002', '3002']

In [30]:
temp = ds['1002']*units.degC
hum=ds['3002']*units.percent

In [31]:
mpcalc.dewpoint_from_relative_humidity(temp, hum)

<xarray.DataArray (time: 505, location: 1)>
<Quantity([[ 2.65749656]
 [ 2.50903096]
 [ 2.86240144]
 [ 6.40354124]
 [ 7.86772804]
 [ 8.3326654 ]
 [ 5.62502216]
 [ 5.82004314]
 [ 5.90751352]
 [ 4.87650363]
 [ 2.71355351]
 [ 1.03941705]
 [ 1.55335417]
 [ 0.25840958]
 [-0.58016805]
 [-1.10462001]
 [ 0.22920189]
 [ 1.77749325]
 [ 2.37524667]
 [ 2.92085593]
...
 [ 6.54206554]
 [ 3.74643889]
 [ 2.03157588]
 [ 1.67686713]
 [ 2.58983461]
 [ 2.67325189]
 [ 0.36387038]
 [ 1.6971744 ]
 [ 1.58845925]
 [ 2.8893772 ]
 [ 1.93106069]
 [ 0.79748725]
 [ 0.96553357]
 [ 5.49480545]
 [ 5.71295151]
 [ 5.77667711]
 [ 8.40592166]
 [ 9.94819274]
 [10.48648144]
 [10.8412255 ]], 'degree_Celsius')>
Coordinates:
  * time      (time) datetime64[ns] 2020-06-12 ... 2020-07-03
  * location  (location) int32 101533